# Vaje 2: Linearni napovedni modeli

## Naloga 1: Linearna regresija

1.a: Ustvari podatkovno množico z vsaj desetimi spremenljivkami, na kateri boš testiral pristop. Funkcija ciljne spremenljivke, ki jo definiraš, naj bo numerična, naj vsebuje šum in ima največ 5 spremenljivk.

<details>
  <summary>Namig:</summary>

  *Pomagaj si s [funkcijo numpy.random.random_sample()](https://numpy.org/doc/1.25/reference/random/generated/numpy.random.random_sample.html) in s [funkcijo numpy.random.normal()](https://numpy.org/doc/1.25/reference/random/generated/numpy.random.normal.html)*.
   
</details>

In [23]:
import numpy as np
import pandas as pd


# velikost vzorca
n = 1000

X = np.random.random_sample((n,10)) 

y = (
    2.5 * X[:, 0] 
    - 1.7 * X[:, 2] 
    + 0.8 * X[:, 4]**2
    + np.sin(2 * np.pi * X[:, 6])     # ker so podatki v [0,1), damo periodično funkcijo
    + 3.0 * np.log(X[:, 9] + 1) 
    + 0.1 * np.random.normal(n)  #šum
)



columns = [f"x{i+1}" for i in range(10)]
df = pd.DataFrame(X, columns=columns)
df["y"] = y



1.b: Podatke razdeli na učno in testno množico. Razmerje med velikostjo učne in testne množice naj bo 4:1.

In [24]:
from sklearn.model_selection import train_test_split

#train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


1.c: Natreniraj model linearne regresije in oceni njegovo točnost na testni množici z RMSE.

<details>
  <summary>Namig:</summary>

  *Uporabi [napovedni model sklearn.linear_model.LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) in [sklearn.metrics.root_mean_squared_error()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html)*.

</details>

In [25]:
from sklearn.linear_model import LinearRegression 

reg = LinearRegression().fit(X_train, y_train)
pred = reg.predict(X_test)

from sklearn.metrics import root_mean_squared_error

RMSE = root_mean_squared_error(y_test, pred) 
RMSE


0.44692948374767516

1.d: Odgovori na naslednja vprašanja:
- Kateri koeficienti spremenljivk pozitivno korelirajo s ciljno spremenljivko in kateri negativno?
- Katera spremenljivka najbolj vpliva na napoved modela?
- Kako bi brez klica metode LinearRegression.predict() ugotovil vrednost, ki bi jo model napovedal za vektor ničel?
- Kolikšna je razlika med napako na učni in testni množici?
- Koliko na točnost vplivajo spremenljivke, ki se v ciljni funkciji ne pojavijo?

In [26]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))

cor = df.corr().style.background_gradient(cmap='coolwarm')
cor

coefs = reg.coef_
print(coefs)

print(f"\nvektor ničel bi imel vrednost: {reg.intercept_:.4f}")


from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(y_train, reg.predict(X_train))
mse_test = mean_squared_error(y_test, reg.predict(X_test))

print(f"\nMSE (train): {mse_train:.4f}")
print(f"MSE (test):  {mse_test:.4f}")
print(f"Razlika:     {abs(mse_train - mse_test):.4f}")

[ 2.63827902  0.00975361 -1.75625929 -0.02100467  0.86450526 -0.0743254
 -1.88687319 -0.07168256 -0.06820006  1.93279612]

vektor ničel bi imel vrednost: 100.8621

MSE (train): 0.2082
MSE (test):  0.1997
Razlika:     0.0085


<Figure size 1000x800 with 0 Axes>

## Naloga 2: Logistična regresija

2.a: Ustvari podatkovno množico z vsaj desetimi spremenljivkami, na kateri boš testiral pristop. Vrednosti ciljne spremenljivke naj bodo diskretne, naj vsebujejo šum in naj bodo definirane z največ 5-imi spremenljivkami.

<details>
  <summary>Namig:</summary>

  *Najprej definiraj funkcijo ciljne spremenljivke, ki bo numerična. Numerične vrednosti nato diskretiziraj.*

</details>

In [27]:
n = 1000

X = np.random.random_sample((n,10)) 

y = (
    2.5 * X[:, 0] 
    - 1.7 * X[:, 2] 
    + 0.8 * X[:, 4]**2
    + np.sin(2 * np.pi * X[:, 6])     # ker so podatki v [0,1), damo periodično funkcijo
    + 3.0 * np.log(X[:, 9] + 1) 
    + 0.1 * np.random.normal(n)  #šum
)

y_d = pd.cut(y, bins=2, labels=['true', 'false'])

columns = [f"x{i+1}" for i in range(10)]
df = pd.DataFrame(X, columns=columns)
df["y"] = y_d

df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,y
0,0.527041,0.285879,0.338183,0.659858,0.982471,0.076392,0.722569,0.020385,0.782828,0.302115,true
1,0.627158,0.961025,0.650158,0.097692,0.185465,0.149553,0.087355,0.995233,0.459677,0.220911,true
2,0.626641,0.503461,0.122321,0.846249,0.245763,0.658295,0.648249,0.991912,0.659397,0.603749,false
3,0.463172,0.639930,0.267017,0.966319,0.220102,0.584191,0.220422,0.531662,0.817508,0.093373,true
4,0.854881,0.075517,0.840727,0.219801,0.624141,0.076994,0.159599,0.663906,0.427680,0.842951,false
...,...,...,...,...,...,...,...,...,...,...,...
995,0.668780,0.423355,0.746432,0.470368,0.380970,0.220031,0.865534,0.036049,0.456383,0.073588,true
996,0.976170,0.706929,0.421577,0.541893,0.914566,0.732941,0.862013,0.107662,0.539317,0.775490,false
997,0.290446,0.163570,0.279508,0.578274,0.651417,0.798303,0.536749,0.549655,0.344469,0.131380,true
998,0.802898,0.811415,0.461369,0.592210,0.018917,0.391390,0.168283,0.686851,0.860506,0.259207,false


2.b: Podatke razdeli na učno in testno množico. Razmerje med velikostjo učne in testne množice naj bo 4:1.

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y_d, test_size = 0.2)

2.c: Preveri razmerje med pozitivnimi (True) in negativnimi (False) vrednostmi ciljne spremenljivke znotraj celotne podatkovne množice, znotraj učne množice in znotraj testne množice. Se razmerja skladajo s tvojim pričakovanjem? Zakaj ja/ne?

In [29]:
print(f"cela:     {sum(y_d =='true')/len(y_d):.4f}")

print(f"učna:     {sum(y_train =='true')/len(y_train):.4f}")

print(f"testna:     {sum(y_test =='true')/len(y_test):.4f}")

cela:     0.5330
učna:     0.5300
testna:     0.5450


2.d: Natreniraj model logistične regresije in oceni njegovo točnost na testni množici z metriko accuracy.

<details>
  <summary>Namig:</summary>

  *Uporabi [napovedni model sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) in [sklearn.metrics.accuracy_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)*.

</details>

In [30]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X_train, y_train)
pred = log_reg.predict(X_test)

from sklearn.metrics import accuracy_score

pred
accuracy = accuracy_score(y_test, pred)
accuracy

0.885

2.e: Odgovori na naslednja vprašanja:
- Kateri koeficienti spremenljivk pozitivno korelirajo s ciljno spremenljivko in kateri negativno?
- Katera spremenljivka najbolj vpliva na napoved modela?
- Kako bi brez klica metode LinearRegression.predict() ugotovil vrednost, ki bi jo model napovedal za vektor ničel?
- Kolikšna je razlika med napako na učni in testni množici?
- Koliko na točnost vplivajo spremenljivke, ki se v ciljni funkciji ne pojavijo?

In [31]:
coefs = log_reg.coef_
print(coefs)

print("Intercept:", log_reg.intercept_)

pred_train= log_reg.predict(X_train)

acc_train = accuracy_score(y_train, pred_train )
acc_test = accuracy_score(y_test, pred)

print(f"\nacc (train): {acc_train:.4f}")
print(f"acc (test):  {acc_test:.4f}")
print(f"Razlika:     {abs(mse_train - mse_test):.4f}")

[[-5.68173355  0.28458329  3.40223176 -0.24141429 -2.24685548 -0.3958299
   4.17601414 -0.51955006 -0.26513708 -4.20995595]]
Intercept: [3.16857064]

acc (train): 0.8688
acc (test):  0.8850
Razlika:     0.0085


2.f: Natreniraj model linearne regresije in ga primerjaj z modelom logistične regresije. Lahko pri napovedih opaziš kaj nenavadnega?

In [32]:
pred_lin = reg.predict(X_test)
y_binary = (pred_lin > np.median(pred_lin))

pred_log = log_reg.predict(X_test)

print("Logistična regresija - prve napovedi:", pred_log[:10])
print("Linearna regresija - prve napovedi:", pred_lin[:10])



Logistična regresija - prve napovedi: ['true' 'false' 'true' 'false' 'true' 'true' 'false' 'false' 'true' 'true']
Linearna regresija - prve napovedi: [ 99.83673669 102.67739211 101.04847429 102.98309147 100.6699106
 100.41169438 102.72250256 102.47234981  99.92288799 100.80792473]


## Bonus: Primerjava linearnih modelov z modelom k-najbližjih sosedov

B.a: Premisli, kakšen vpliv ima domena/razpon podatkov na linearne modele in kakšnega na model k-najbližjih sosedov.

**Linearni modeli** so invariantni na linearne transformacije spremenljivk (če standardiziraš, dodaš konstanto, spremeniš mersko enoto):

Vpliv razpona je torej zanemarljiv za napoved z linearnim modelom, a pomemben za interpretacijo koeficientov (veliki razponi → majhni koeficienti in obratno).

**k-NN** je razdaljni model: odloča na podlagi evklidske (ali druge) razdalje med točkami.

Če ena spremenljivka ima večji razpon kot druge, bo dominirala razdaljo.

Zato pri k-NN (in nasploh pri razdaljnih modelih) vedno standardiziramo ali normaliziramo podatke, da imajo vse spremenljivke primerljivo težo.

B.b: Primerjaj točnost linearne regresije s točnostjo modela k-najbližjih sosedov pri napovedovanju numeričnih spremenljivk. Uporabi kodo iz naloge 1.

In [34]:
from sklearn.neighbors import KNeighborsRegressor

#uporabimo y ko ta ni diskretiziran
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Instanciramo model
knn = KNeighborsRegressor(5)
# Naučimo model

knn_lin = knn.fit(X_train, y_train)

rmse_test_knn = root_mean_squared_error(y_test, knn_lin.predict(X_test))
print('rmse za knn', rmse_test_knn)
print('rmse za lin reg', RMSE)

rmse za knn 0.6131886437742833
rmse za lin reg 0.44692948374767516


B.c: Primerjaj točnost logistične regresije s točnostjo modela k-najbližjih sosedov pri napovedovanju diskretnih spremenljivk. Uporabi kodo iz naloge 2.

In [37]:
from sklearn.neighbors import KNeighborsClassifier

#uporabimo y ko ta ni diskretiziran
X_train, X_test, y_train, y_test = train_test_split(X, y_d, test_size = 0.2)

# Instanciramo model
knn_class = KNeighborsClassifier(5)
# Naučimo model

knn_fit_class = knn_class.fit(X_train, y_train)
pred = knn_fit_class.predict(X_test)

acc_test_knn = accuracy_score(y_test, pred)

print('acc za knn', acc_test_knn)
print('acc za log reg', accuracy)

acc za knn 0.805
acc za log reg 0.885


B.d: Najdi omejitev, ki se pojavi pri linearni regresiji in se ne pojavi pri modelu k-najbližjih sosedov in omejitev, ki se pojavi pri modelu k-najbližjih sosedov in se ne pojavi pri linearni regresiji.

Linearni model – omejitev: predpostavlja linearno zvezo med značilkami in ciljem (slabo za nelinearne odnose).

k-NN – omejitev: visoka računska zahtevnost in problem dimenzionalnosti (slabo za velike podatke in veliko število značilk).